# Model for Credit Default prediction

Author: sharon

In [6]:
# Data Handling
import pandas as pd
import numpy as np

# Ploting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# System
import os
from os import path
import sys
import warnings
warnings.filterwarnings('ignore')

# Modelling
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

from sklearn.impute import SimpleImputer
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score, f1_score, confusion_matrix
import shap

For the model the steps are as follow

1. Preprocessing:
    1. One Hot Encoding for categorical variables
    2. Standardizes the data for PCA


2. SMOTE oversampling to handle class imbalance:
    - SMOTE generates synthetic samples for the minority class to balance the dataset

3. Model Trainig :
    1. Here an XGBoost model will be used directly, but more algorithm can be tested.
    2. Hyperparameters tuning using GridSearchCV
    3. Evaluation metrics

4. Model Interpretation using SHAP values

In [7]:
# Define Variables for files and paths

MAIN_DIR = path.dirname(__vsc_ipynb_file__)
OUTPUT_DIR = path.join(MAIN_DIR,"output")
INPUT_DIR = path.join(MAIN_DIR, "data")

DF_PATH = path.join(OUTPUT_DIR, "df_train_test.csv")

In [8]:
# import functions from utils.py

sys.path.append(MAIN_DIR)
from utils import *

# Preprocess Data

- **Missing data**: In this step the NA values are actually processed
    - Drop columns with more than 65% of NA values. At first they were not drop because in the EDA it is usefult to understand NA patterns.
    - XGBoost handle natively missing data, but some simple imputations could be done

- **Encoding**: One hot Encoding

    - When there is categorical data, it has to be consideren how to handle it, if applying One-hot encoding or label encoding, depending on the nature of the variable. Also, its important to consider the dimensionality of the dataset, when feature space grows (this can happen with OHE) and lead to the curse of dimensionality.

- **Resampling**: XGBoost handles class imbalance natively by setting the parameter scale_pos_weight. But if necessary resampling techiques might by used 
    -Avoids creating synthetic samples, which can sometimes introduce noise.



In [9]:
# Read data with some new features and tranformations done in EDA.ipynb
df = pd.read_csv(DF_PATH).drop('uuid', axis = 1)
X = df.drop(columns = ['default']).copy()
y = df['default']

categorical_columns = X.select_dtypes(include=['object', 'category']).columns

#X[categorical_columns] = X[categorical_columns].astype('category')

# Split data
X_train_comp, X_test_comp, y_train, y_test = train_test_split(X, y, test_size=0.215, random_state=42, stratify=y)

# compute NA counts on training set to avoid data leakage
print('\nNA counts in training set')
na_counts = count_na(X_train_comp, verbose=True)# Get na proportions per columns
na_drop = na_counts.index[na_counts>65].to_list() # Drop columns that have more than 80% of 

print('\ncolumns to drop')
print('\n'.join(na_drop))

# Drop columns with more than 65% NA
X_train = X_train_comp.drop(columns=na_drop)
X_test = X_test_comp.drop(columns=na_drop)

# Preprocessing

categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns
numerical_columns = X_train.select_dtypes(include=['number']).columns

# Imputation for numerical 
num_imputer = SimpleImputer(strategy='mean')
X_train[numerical_columns] = num_imputer.fit_transform(X_train[numerical_columns])
X_test[numerical_columns] = num_imputer.fit_transform(X_test[numerical_columns])


# Imputation for categorical
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[categorical_columns] = cat_imputer.fit_transform(X_train[categorical_columns])
X_test[categorical_columns] = cat_imputer.fit_transform(X_test[categorical_columns])

# Encoding
X_train = pd.get_dummies(X_train, columns=categorical_columns, dtype=int)
X_test = pd.get_dummies(X_test, columns=categorical_columns, dtype=int)


print("\nAfter preprocessing")
print('Train set shape: ', X_train.shape)
print('Test set shape: ', X_test.shape)



NA counts in training set
Number of columns with missing values: 256

columns to drop
person_bith_year_month_1_no_aggregation
client_type_1_month_9_no_aggregation
plan_data_month_12_no_aggregation
billing_pattern_03_last_12_months_mean
line_balance_type_01_month_1_sum
billing_pattern_08_last_12_months_mean
line_balance_type_07_month_1_sum
line_balance_type_04_last_6_months_max

After preprocessing
Train set shape:  (14604, 261)
Test set shape:  (4001, 261)


# Fit XGBoost wit all features

First we are going to fit the xgboost model with all the variables and use SHAP values to select the most relevant features.

In [10]:

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "eta": 0.01,
    "scale_pos_weight":len(y_train[y_train == 0]) / len(y_train[y_train == 1]),
    "objective": "binary:logistic",
    "subsample": 0.5,
    "base_score": np.mean(y_train),
    "eval_metric": "logloss",
}
model = xgb.train(
    params,
    dtrain,
    5000,
    evals=[(dtest, "test")],
    verbose_eval=100,
    early_stopping_rounds=20
)

[0]	test-logloss:0.45618
[41]	test-logloss:0.45065


Select features to keep using shap values feature importances, and keeping the variables that accumulate 95% of the total importance.

In [13]:
# Initialize Shap values
explainer = shap.TreeExplainer(model)
explanation = explainer(dtest)

In [14]:

# Get feature Importances
importances = []
for i in range(explanation.values.shape[1]):
    importances.append(np.mean(np.abs(explanation.values[:, i])))
    
feature_importances = {fea: imp for imp, fea in zip(importances, dtrain.feature_names)}

# Keep features that accumluate 95% of the total importance
cumsum = np.cumsum(sorted(importances, reverse=True))
threshold_idx = np.searchsorted(cumsum, 0.95 * cumsum[-1])
threshold = sorted(importances, reverse=True)[threshold_idx]

selected_features = [fea for fea, imp in feature_importances.items() if imp>threshold]

len(selected_features)


165

# Model: XGBoost with HP tuning

First tune tree parameters

In [20]:
# cross-validation 
kfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Parameter grid 
param_test1 = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2),
    'gamma': [i/10.0 for i in range(0, 5)],
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
    'subsample':[i/100.0 for i in range(75,90,5)],
    'colsample_bytree':[i/100.0 for i in range(75,90,5)],
    'learning_rate':[0.001, 0.05, 0.1, 0.2, 0.3]
}


xgb1 = XGBClassifier(
    n_estimators=140,
    objective='binary:logistic',
    n_jobs=4,  
    scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]),
    random_state=27 
)

# Grid search setup
gsearch1 = GridSearchCV(
    estimator=xgb1,
    param_grid=param_test1,
    scoring='f1',
    n_jobs=4,
    cv=kfolds  
)

# Fit the model
gsearch1.fit(X_train[selected_features], y_train)

# Print the results
print("Best Parameters:", gsearch1.best_params_)
print("Best F1 Score:", gsearch1.best_score_)

KeyboardInterrupt: 

In [ ]:
import json

path = os.path.join(OUTPUT_DIR, 'gridsearch_best_params.json')
with open(path, 'w') as f:
    json.dump(gsearch1.best_params_, f, indent=4)

# Evaluate the model

In [ ]:
best_model = gsearch1.best_predictor_

y_pred = best_model.predict(X_test)
y_pred_prob = best_model.predict_proba(X_test)[:, 1]


print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC Score:", roc_auc_score(y_test, y_pred_prob))



In [ ]:
# confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)


g = sns.heatmap(conf_mat,  annot=True, cmap=plt.cm.copper, fmt='0.0f')
g.set_title("Confusion Matrix", fontsize=14)
g.set_xticklabels(['', ''], fontsize=14, rotation=90)
g.set_yticklabels(['', ''], fontsize=14, rotation=360)


# Feature Importance

In [ ]:
# Step 7: Model interpretability with SHAP
explainer = shap.Explainer(best_model)
shap_values = explainer(X_test)

# Plot SHAP summary
shap.summary_plot(shap_values, X_test  )

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.waterfall(shap_values)

In [ ]:
for name in selected_features:
    shap.dependence_plot(name, shap_values, X_test)